# Lungs Segmentation
The segmentation of lungs may, possibly, proceed directly in 3D as follows:
1. Run binarization of the CT image using a threshold of -320 HU – every voxel
with HU lower than this threshold should receive label 1 (air label) and the
remaining voxels should receive label 0
2. Use body mask to select only air regions within body
3. Design a sequence of morphological (and other appropriate) operations to fill
the holes in the interior of lungs and to remove ‘air’ clusters which do not
correspond to lungs (e.g. gas in bowels) – at the end one should be left with
clusters which correspond only to airways
4. Use watershed from markers (scikit-image -> segmentation -> watershed) to
extract the left and the right lung from the segmentation being the result of step
(3) above. Before using watershed design a procedure for defining the three
markers (marker of left lung, marker of right lung, marker of background).
5. To compare segmentation results with reference segmentations available at
Lab One Drive use Dice coefficient and Hausdorff distance (find the definitions of
these   quantities)   as   implemented   in   surface-distance   [package](https://github.com/google-deepmind/surface-distance).
The project results (Dice coefficients and Hausdorff distance) should be
reported   for   the   three   tasks:   body   mask   segmentation,   left   lung
segmentation, right lung segmentation.


## Imports

In [19]:
import numpy as np
import nibabel as nib
import cv2
from skimage import morphology, measure
from skimage.segmentation import watershed
from skimage import filters
from scipy import ndimage
import matplotlib.pyplot as plt
# from pycimg import CImg

import numpy as np
from skimage.measure import label, regionprops
from skimage.morphology import reconstruction


TODO
#body
Body:
binarization
opening
flood_fill

#binarize lungs
Do histogram of the image
do kmeans to get threashold
do binarization with threashold

#lung segmentation


## Load / save .nii files and Visualization

In [20]:
def load_nii_gz_file(file_path: str) -> tuple:
    nii_img = nib.load(file_path)
    nii_data = nii_img.get_fdata()
    return nii_data, nii_img.affine

def save_to_nii(segmented_data: np.ndarray, affine: np.ndarray, output_path: str) -> None:
    segmented_nii = nib.Nifti1Image(segmented_data.astype(np.uint8), affine)
    nib.save(segmented_nii, output_path)
    
def view_nii_data(nii_data: np.ndarray) -> None:
    for i in range(nii_data.shape[2]):
        cv2.imshow('slice', nii_data[:, :, i])
        cv2.waitKey(0)
    cv2.destroyAllWindows()
    
def visualize_photos(original: np.ndarray, segmented: np.ndarray, reference: np.ndarray, *slices: int) -> None:
    num_slices = len(slices)
    plt.figure(figsize=(15, 5 * num_slices))  # Adjust figure size based on the number of slices

    for i, slice_num in enumerate(slices):
        # Original slice
        plt.subplot(num_slices, 3, 3 * i + 1)
        plt.title(f"Original Slice {slice_num}")
        plt.imshow(original[:, :, slice_num], cmap="gray")
        
        # Segmented slice
        plt.subplot(num_slices, 3, 3 * i + 2)
        plt.title(f"Segmented Slice {slice_num}")
        plt.imshow(segmented[:, :, slice_num], cmap="gray")
        
        # Reference slice
        plt.subplot(num_slices, 3, 3 * i + 3)
        plt.title(f"Reference Slice {slice_num}")
        plt.imshow(reference[:, :, slice_num], cmap="gray")

    plt.tight_layout() 
    plt.show()

## 1. Body differentiation

In [21]:
class Transforms:
    def __init__(self):
        pass
        
    def binarize_images(self, image: np.ndarray, threshold: float = -320, use_otsu: bool = False) -> np.ndarray:
        threshold = filters.threshold_otsu(image) if use_otsu else threshold
        image = np.where(image < threshold, 1, 0)
        return image.astype(np.uint8)
    
    def binarize_images_kmeans(self, image: np.ndarray, max_iter: int = 100) -> float:
        min_val = image.min()
        not_background = image > min_val
        img_not_background = image * not_background
        
        threshold = img_not_background.sum() / not_background.sum()
        prev_threshold = 0
        
        plt.imshow(not_background[..., 70], cmap='gray')
        plt.show()
        
        for _ in range(max_iter):
            foreground_mask = image > threshold
            background_mask = image <= threshold
            
            foreground = img_not_background * foreground_mask
            background = img_not_background * background_mask
            
            avg_foreground = foreground.sum() / foreground_mask.sum()
            avg_background = background.sum() / background_mask.sum()
            
            new_threshold = (avg_foreground + avg_background) / 2
            
            if abs(new_threshold - prev_threshold) < 0.1:
                threshold = new_threshold
                break
            
            prev_threshold = threshold
            threshold = new_threshold
        
        print(f"threshold: {threshold}")
            
        image = (image > threshold) * not_background
        
        plt.imshow(image[..., 70], cmap='gray')
        plt.show()
        
        return image.astype(np.uint8)

In [22]:
from skimage.segmentation import flood
class BodyMask(Transforms):
    def __init__(self, image: np.ndarray) -> None:
        self.img: np.ndarray = image
        
    def initial_transform(self) -> np.ndarray:
        binary_image = self.binarize_images_kmeans(self.img)
        opened_img = morphology.binary_opening(binary_image, morphology.ball(2))
        self.img = opened_img
        
    
    def reconstruct_3d(self) -> np.ndarray:
        seed = np.zeros_like(mask)
        list_of_corners = [(0, 0, 0), (0, 0, -1), (0, -1, 0), (-1, 0, 0), (-1, -1, -1), (-1, -1, 0), (-1, 0, -1), (0, -1, -1)]
        for corner in list_of_corners:
            seed[corner] = 1
            
        labels = label(self.img, connectivity=1)
        regions = regionprops(labels)
        largest_region = max(regions, key=lambda x: x.area)
        largest_region_mask = labels == largest_region.label
        
        mask = largest_region_mask
        neg_mask = np.logical_not(mask)
        
        reconstructed = reconstruction(seed, neg_mask)
        neg_reconstructed = 1 - reconstructed
        
        return neg_reconstructed
    
    def flood_fill_3d(self, seed_point: tuple = (0, 0, 0)) -> np.ndarray:
        labels = label(self.img, connectivity=1)
        regions = regionprops(labels)
        largest_region = max(regions, key=lambda x: x.area)
        
        neg_mask = (labels == largest_region.label) * 1
        mask = flood(neg_mask, seed_point)
        mask = np.logical_not(mask)
        return mask


## Main

In [23]:
def main(img_path: str,reference_img_path: str, body_mask_path: str, output_path: str, ) -> None:
    # load images
    img, affine = load_nii_gz_file(img_path)
    body_masks, _ = load_nii_gz_file(body_mask_path)
    reference_img, _ = load_nii_gz_file(reference_img_path)

    # transform
    body = BodyMask(img)
    body.initial_transform()
    body_mask = body.reconstruct_3d()
    
    visualize_photos(body_mask, body_masks, abs(body_mask - body_masks), 50, 70, 100, 120)
    # print("Dice_coefficient, Hausdorff_distance", calculate_score(imgs_transform, reference_img))

## Run code

In [ ]:
main('Images/IMG_0001.nii.gz','ReferenceSegmentations/LUNGS_IMG_0001.nii.gz', 'BodyMasks/BODYMASK_IMG_0001.nii.gz', 'output.nii.gz')
